# Image Recommendation System via Collaborative Filtering

# ***Please read the instructions very carefully***

1.   Assignment must be implemented in Python 3 only.
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for evaluation metrics, data visualization (matplotlib etc.).
3.   You will be evaluated not just on the overall performance of the model and also on the experimentation with hyper parameters, data prepossessing techniques etc.
4.   ⚠️ The Assignment will be evaluated automatically. Please adhere to taking proper inputs from `config.csv` file. You can change your `config.csv` file to experiment with your code. But at the end, make sure that your outputs are corresponding to input values in `config.csv`
5.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

## About the Dataset
Behance is a community art website where users showcase and discover creative work. Each user is able to “appreciate” (equivalent to a “like” on Instagram or a “react” on Facebook) an image, indicating that they like the image. It is in the website’s best interests to show users pictures that they would like, to keep them engaged for longer. For this question, given a set of pictures that a user has already appreciated, you have to show them a new picture that they would like based on what similar users appreciated.


<br><br>
**The dataset has information of 1 million appreciates of 63,497 users on 178,788 items. The file Behance appreciate 1M has a triplet in each line in the form of (user id, item id, unix timestamp).**

**Task: Take the inputs from the config.csv file and output the recommendations for a particular person**
- Collaborative Filtering is a way to predict items to the user based on the the
user’s history and the history of similar users. The similarity between users can be quantified by the number of images that both the users appreciated.
- The images appreciated by a similar user would be the most suitable images to show a user. Since we can find the similarity between any two users, we would be able to find the “nearest” neighbours of any user, allowing us to use a KNN-based algorithm to recommend new images to a user.
- Since people do not like seeing pictures that they have seen already. Make sure that you do not recommend pictures that a user has appreciated already.
- Output the final response will be saved in the file named ```config['output_file']```.


**Output file format:**
Populate the output file with images that the user has not seen of the k most
similar users, in descending order of their similarity. Each line in the output
file should be a duplet in the form of (item id, user id), where the user id is the
id of the kth similar user. The order of the images corresponding to the same
similar user would not matter. The output file would look something like this:
```
item_id_1_of_1st_similar_user 1st_most_similar_user_id
item_id_2_of_1st_similar_user 1st_most_similar_user_id
item_id_3_of_1st_similar_user 1st_most_similar_user_id
...
item_id_1_of_2nd_similar_user 2nd_most_similar_user_id
item_id_2_of_2nd_similar_user 2nd_most_similar_user_id
item_id_3_of_2nd_similar_user 2nd_most_similar_user_id
...
item_id_1_of_kth_similar_user kth_most_similar_user_id
item_id_2_of_kth_similar_user kth_most_similar_user_id
item_id_3_of_kth_similar_user kth_most_similar_user_id
```

The dataset was extracted using Behance’s API as a part of the paper
“Vista: A visually, socially, and temporally-aware model for artistic
recommendation, RecSys, 2016”. Check out this [Google Drive folder](https://drive.google.com/drive/folders/0B9Ck8jw-TZUEc3NlMjVXdDlPU1k?resourcekey=0-6_8ykn0o4fLc5fuTEm91xA) for
more information about the dataset.


Have fun! The users are waiting to see new pictures!

### Import necessary libraries

In [ ]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD

In [ ]:
# Config Generation Sample Code.
# ⚠️ Only for experimentation on your side.
# ⚠️ Should be commented during submission.

# df = pd.DataFrame(data=[{'id':276633,
#                   'k':5,
#                   'dataset_file':'./Behance_appreciate_1M',
#                   'output_file':'./output.txt'}])
# df.to_csv('config.csv')

In [ ]:
config = pd.read_csv('config.csv').iloc[0]

In [ ]:
config

Unnamed: 0                            0
id                               276633
k                                     5
dataset_file    ./Behance_appreciate_1M
output_file                ./output.txt
Name: 0, dtype: object

In [ ]:
user = config['id']
k_value = config['k']

### Read the Data

In [ ]:
with open(config['dataset_file'], 'r') as inFile:
    appreciate_data = inFile.readlines()

In [ ]:
# your code here

### Initialize a dictionary to store the item_ids that a user likes

### Go through each line of the input file and construct the user_likes dictionary

In [ ]:
user_likes = dict()

In [ ]:
for line in appreciate_data:
    line = line.strip()
    
    user_id = int(line.split()[0])
    item_id = int(line.split()[1])

    if user_id not in user_likes:
        user_likes[user_id] = list()
    
    user_likes[user_id].append(item_id)

In [ ]:
rows=[] #user_id
cols=[] #item_id
data=[] #boolean_value
for line in appreciate_data:
  line=line.strip()
  user_id = int(line.split()[0])
  item_id = int(line.split()[1])
  rows.append(user_id)
  cols.append(item_id)
  data.append(1)

In [ ]:
user_id_set=set(rows)
rows_len=len(set(rows))
item_id_set=set(cols)
cols_len=len(set(cols))
# print(len(user_id_set),len(item_id_set))
# print(rows_len,cols_len)
user_id_dict=dict()
item_id_dict=dict()
user_index=0
for i in user_id_set:
  user_id_dict[i]=user_index
  user_index+=1
item_index=0
for j in item_id_set:
  item_id_dict[j]=item_index
  item_index+=1
user_id_set.clear()
item_id_set.clear()


63497 178788
63497 178788


In [ ]:
rows_mapped=[user_id_dict[i] for i in rows]
cols_mapped=[item_id_dict[j] for j in cols]
# print(rows,cols)
# print(rows_mapped,cols_mapped)
print(len(rows),len(rows_mapped))
print(len(cols),len(cols_mapped))


1000000 1000000
1000000 1000000


In [ ]:
data_csr=csr_matrix((data, (rows_mapped, cols_mapped)), shape=(rows_len,cols_len))

In [ ]:
data_csr.shape

(63497, 178788)

In [ ]:
svd = TruncatedSVD(n_components=2000, n_iter=7, random_state=42)
truncated_csr=svd.fit_transform(data_csr)

In [ ]:
print(type(truncated_csr))
truncated_csr.shape

<class 'numpy.ndarray'>


(63497, 2000)

In [ ]:
var_explained = svd.explained_variance_ratio_.sum()
print(var_explained)

0.5927619974021215


### Use KNN after Collaborative Filtering to find nearest neighbors

In [ ]:
def euclidean_dist(point1,point2):
  return np.linalg.norm(point1 - point2)

In [ ]:
# your code here
# your code here
def neighbors(user,k_value):
    """ returns an iterable object (like list or generator) """
    mapped_user=user_id_dict[user]
    distances=[]
    # mapped_user_norm=norm(truncated_csr[mapped_user])
    for i in range(len(truncated_csr)):
      if(i!=mapped_user):
        # cos_sim = dot(truncated_csr[i],truncated_csr[mapped_user])/(mapped_user_norm*norm(truncated_csr[i]))
        dist=euclidean_dist(truncated_csr[i],truncated_csr[mapped_user])
        distances.append([dist,i])
    print(len(distances))
    distances.sort(key=lambda x:x[0])
    mapped_neighbour_users=[distances[i][1] for i in range(k_value)]
    neighbour_users=[]
    for key,value in user_id_dict.items():
      if value in mapped_neighbour_users:
        neighbour_users.append(key)
    print(neighbour_users)
    return neighbour_users 

In [ ]:
neighbour_users=neighbors(user,k_value)
# print(user_likes[neighbour_users[0]])
# print(user_likes[neighbour_users[-2]])

### Open the output file to write all the lines to the file

In [ ]:
input_user_likes=set(user_likes[user])
outFile = open(config['output_file'], 'w')
for n_user in neighbors(user,k_value):
  items=set(user_likes[n_user]).difference(input_user_likes)
  input_user_likes=input_user_likes.union(set(user_likes[n_user]))
  for item_id in items:
    outFile.write(str(item_id) + ' ' + str(n_user) + '\n')
outFile.close()

63496
[148706, 1459742, 1480429, 1525134, 2452817]
